## Model

## Import Packages

In [15]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import statsmodels.api as sm
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score
from statsmodels.stats.outliers_influence import variance_inflation_factor
from lifelines import KaplanMeierFitter

## Import Data

In [16]:
df = pd.read_csv("IWC_Work_Orders_Extract.csv", low_memory=False)

In [17]:
df = df[df['FUNCTIONAL_LOC'].isna() == False]

In [18]:
#filling in missing 'EQUIP_START_UP_DATE' values that already have a corresponding value based on 'FUNCTIONAL_LOC'
df['EQUIP_START_UP_DATE'] = df.groupby('FUNCTIONAL_LOC')['EQUIP_START_UP_DATE'].transform(lambda group: group.ffill().bfill())
#transforming date columns into datetime datatype
df['EQUIP_START_UP_DATE'] = pd.to_datetime(df['EQUIP_START_UP_DATE'], errors='coerce')
df['EXECUTION_START_DATE'] = pd.to_datetime(df['EXECUTION_START_DATE'])
#creating a year column
df['YEAR'] = df['EXECUTION_START_DATE'].dt.year
#seperating out 'FUNCTIONAL_LOC'
df[['SEGMENT_1', 'SEGMENT_2', 'SEGMENT_3', 'SEGMENT_4', 'SEGMENT_5', 'SEGMENT_6']] = df['FUNCTIONAL_LOC'].str.split('-', expand=True, n=5)
#aranging dataset in ascending order of the below features
df = df.sort_values(by=['FUNCTIONAL_LOC','EXECUTION_START_DATE','ACTUAL_START_TIME'], ascending=[True,True,True])

C:\Users\sarob\AppData\Local\Temp\ipykernel_27208\3288744948.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['EQUIP_START_UP_DATE'] = df.groupby('FUNCTIONAL_LOC')['EQUIP_START_UP_DATE'].transform(lambda group: group.ffill().bfill())


In [19]:
#clean 'ACTUAL_START_TIME' by removing milliseconds (if they exist) and convert to datetime time format
df['ACTUAL_START_TIME'] = pd.to_datetime(df['ACTUAL_START_TIME'].str.split('.').str[0], format='%H:%M:%S').dt.time

#combine 'EXECUTION_START_DATE' and 'ACTUAL_START_TIME' into a single datetime column
df['Maintenance_Start_Datetime'] = pd.to_datetime(df['EXECUTION_START_DATE'].astype(str) + ' ' + df['ACTUAL_START_TIME'].astype(str))

#sort the DataFrame by 'FUNCTIONAL_LOC' and 'Maintenance_Start_Datetime'
df = df.sort_values(by=['FUNCTIONAL_LOC', 'Maintenance_Start_Datetime'], ascending=[True, True])

#create a new column to store the time until the next unplanned maintenance
df['Time_To_Failure'] = None

#loop through each machine group
for loc, group in df.groupby('FUNCTIONAL_LOC'):
    #create a variable to track the next unplanned maintenance date
    next_unplanned_date = None

    #loop over the rows in this group
    for idx in reversed(group.index):
        row = df.loc[idx]

        #if the row represents an "Unplanned" maintenance update next_unplanned_date
        if row['MAINTENANCE_ACTIVITY_TYPE'] == 'Unplanned':
            if next_unplanned_date is not None:
                #calculate the time until the next unplanned maintenance
                time_to_failure = (next_unplanned_date - row['Maintenance_Start_Datetime']).days
                df.at[idx, 'Time_To_Failure'] = time_to_failure
            next_unplanned_date = row['Maintenance_Start_Datetime']
        else:
            #for planned maintenance, calculate the time until the next unplanned maintenance
            if next_unplanned_date is not None:
                time_to_failure = (next_unplanned_date - row['Maintenance_Start_Datetime']).days
                df.at[idx, 'Time_To_Failure'] = time_to_failure

#convert the new column into a integer
df['Time_To_Failure'] = pd.to_numeric(df['Time_To_Failure'], errors='coerce').astype('Int64')

In [20]:
#filter for missing values
df_filtered = df[df['Time_To_Failure'].isna() == False]
#find the mean of 'Time_To_Failure' for each machine group
overall_mean = df_filtered['Time_To_Failure'].mean()
#round it into an integer
overall_mean = overall_mean.round()
#fill in missing values
df['Time_To_Failure'].fillna(overall_mean, inplace=True)

C:\Users\sarob\AppData\Local\Temp\ipykernel_27208\3225125504.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Time_To_Failure'].fillna(overall_mean, inplace=True)


In [21]:
#create a new column to store the days since last planned maintenance
df['Days_Since_Planned_Maintenance'] = None

#loop through each machine group
for loc, group in df.groupby('FUNCTIONAL_LOC'):
    #track the last planned maintenance date
    last_planned_date = None

    #loop over the rows in this group
    for idx, row in group.iterrows():
        #if the row represents a "Planned" maintenance, update last_planned_date
        if row['MAINTENANCE_ACTIVITY_TYPE'] == 'Planned':
            last_planned_date = row['Maintenance_Start_Datetime']
            df.at[idx, 'Days_Since_Planned_Maintenance'] = 0  #set to 0 on the day of planned maintenance
        else:
            #for non-planned maintenance, calculate days since the last planned maintenance
            if last_planned_date is not None:
                days_since = (row['Maintenance_Start_Datetime'] - last_planned_date).days
                df.at[idx, 'Days_Since_Planned_Maintenance'] = days_since

#convert the new column to integer
df['Days_Since_Planned_Maintenance'] = pd.to_numeric(df['Days_Since_Planned_Maintenance'], errors='coerce').astype('Int64')

In [22]:
#filter for missing values
df_filtered = df[df['Days_Since_Planned_Maintenance'].isna() == False]
#find the mean of 'Days_Since_Planned_Maintenance' for each machine group
overall_mean = df_filtered['Days_Since_Planned_Maintenance'].mean()
#round it into an integer
overall_mean = overall_mean.round()
#fill in missing values
df['Days_Since_Planned_Maintenance'].fillna(overall_mean, inplace=True)

C:\Users\sarob\AppData\Local\Temp\ipykernel_27208\172671851.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Days_Since_Planned_Maintenance'].fillna(overall_mean, inplace=True)


In [23]:
#create a binary flag for "Unplanned" maintenance
df['Unplanned_Flag'] = (df['MAINTENANCE_ACTIVITY_TYPE'] == 'Unplanned').astype(int)

#convert 'EXECUTION_START_DATE' to datetime if it's not already
df['EXECUTION_START_DATE'] = pd.to_datetime(df['EXECUTION_START_DATE'])

#sort the DataFrame by 'FUNCTIONAL_LOC' and 'EXECUTION_START_DATE'
df = df.sort_values(by=['FUNCTIONAL_LOC', 'EXECUTION_START_DATE'], ascending=[True, True])

#define a rolling window function
def calculate_rolling_unplanned(df, window_size):
    return df.groupby('FUNCTIONAL_LOC')['Unplanned_Flag'].rolling(window=window_size, min_periods=1).sum().reset_index(level=0, drop=True)

#calculate rolling sums for different time windows
days_in_month = 30

df['Unplanned_Rolling_12M'] = calculate_rolling_unplanned(df, window_size=12 * days_in_month)
df['Unplanned_Rolling_6M'] = calculate_rolling_unplanned(df, window_size=6 * days_in_month)
df['Unplanned_Rolling_1M'] = calculate_rolling_unplanned(df, window_size=1 * days_in_month)

In [24]:
#create a binary flag for "Planned" maintenance
df['Planned_Flag'] = (df['MAINTENANCE_ACTIVITY_TYPE'] == 'Planned').astype(int)

#convert 'EXECUTION_START_DATE' to datetime if it's not already
df['EXECUTION_START_DATE'] = pd.to_datetime(df['EXECUTION_START_DATE'])

#sort the DataFrame by 'FUNCTIONAL_LOC' and 'EXECUTION_START_DATE'
df = df.sort_values(by=['FUNCTIONAL_LOC', 'EXECUTION_START_DATE'], ascending=[True, True])

#define a rolling window function
def calculate_rolling_unplanned(df, window_size):
    return df.groupby('FUNCTIONAL_LOC')['Planned_Flag'].rolling(window=window_size, min_periods=1).sum().reset_index(level=0, drop=True)

#calculate rolling sums for different time windows
days_in_month = 30

df['Planned_Rolling_12M'] = calculate_rolling_unplanned(df, window_size=12 * days_in_month)
df['Planned_Rolling_6M'] = calculate_rolling_unplanned(df, window_size=6 * days_in_month)
df['Planned_Rolling_1M'] = calculate_rolling_unplanned(df, window_size=1 * days_in_month)

In [25]:
#create a binary flag for each equipment occurrence
df['Equipment_Occurrence_Flag'] = 1

#group by 'FUNCTIONAL_LOC' and 'EQUIPMENT_ID', perform cumulative sum to get rolling count
df['Cumulative_Equipment_Replacements'] = df.groupby(['FUNCTIONAL_LOC', 'EQUIPMENT_ID'])['Equipment_Occurrence_Flag'].cumsum()

In [26]:
X = df.drop(columns=['Equipment_Occurrence_Flag','YEAR','EQUIPMENT_DESC',	'EQUIP_CAT_DESC','EQUIP_START_UP_DATE',	'EQUIP_VALID_FROM',	'EQUIP_VALID_TO',
'FUNCTIONAL_LOC',	'FUNCTIONAL_AREA_NODE_1_MODIFIED',	'FUNCTIONAL_AREA_NODE_2_MODIFIED',	'FUNCTIONAL_AREA_NODE_3_MODIFIED',	'FUNCTIONAL_AREA_NODE_4_MODIFIED', 'FUNCTIONAL_AREA_NODE_5_MODIFIED',
'MAINTENANCE_ACTIVITY_TYPE',	'ORDER_DESCRIPTION',	'MAINTENANCE_TYPE_DESCRIPTION','MAINTENANCE_PLAN','ORDER_ID',	'PLANT_ID',	'PRODUCTION_LOCATION',	'EXECUTION_START_DATE',	'EXECUTION_FINISH_DATE',
'ACTUAL_START_TIME',	'ACTUAL_FINISH_TIME',	'SEGMENT_1',	'SEGMENT_2',	'SEGMENT_3',	'SEGMENT_4',	'SEGMENT_5',	'SEGMENT_6'])

In [27]:
df = df.drop(columns=['Unplanned_Rolling_12M',	'Unplanned_Rolling_6M','Planned_Rolling_12M',	'Planned_Rolling_6M','Equipment_Occurrence_Flag','Unplanned_Flag','Planned_Flag','YEAR'])

In [28]:
df_unplanned = df[df['MAINTENANCE_ACTIVITY_TYPE']=='Unplanned']

In [29]:
# Split FUNCTIONAL_LOC into multiple components based on the '-' delimiter, allowing up to 5 columns
df_split = df_unplanned['FUNCTIONAL_LOC'].str.split('-', expand=True, n=5)

# If the resulting split produces fewer than 5 columns, fill with NaN and rename accordingly
df_split = df_split.iloc[:, :5]  # Ensure we have exactly 5 columns, adding NaN if necessary

# Rename the columns (ensure we have 5 column names to match the 5 split columns)
df_split.columns = ['Plant', 'Process', 'Subprocess', 'Product_Line', 'Machine']

# Concatenate the new split columns back to the original dataframe
df_unplanned = pd.concat([df_unplanned, df_split], axis=1)

In [30]:
df_unplanned.head()

,ORDER_ID,PLANT_ID,PRODUCTION_LOCATION,EXECUTION_START_DATE,EXECUTION_FINISH_DATE,ACTUAL_START_TIME,ACTUAL_FINISH_TIME,ACTUAL_WORK_IN_MINUTES,MAINTENANCE_PLAN,MAINTENANCE_ITEM,...,Time_To_Failure,Days_Since_Planned_Maintenance,Unplanned_Rolling_1M,Planned_Rolling_1M,Cumulative_Equipment_Replacements,Plant,Process,Subprocess,Product_Line,Machine
1016872,702711887,G221,SUZUKA,2021-01-16,2021-01-16,05:50:04,08:00:00.000,360.0,NaN,NaN,...,40,4,1.0,29.0,791,G221,CLR,A85,E06,None
314204,700108001,G221,SUZUKA,2017-07-25,2017-07-25,07:00:00,07:00:00.000,45.0,NaN,NaN,...,343,10,1.0,0.0,1,G221,PRD,None,None,None
973849,700702956,G221,SUZUKA,2018-07-03,2018-07-03,07:00:00,08:50:09.000,270.0,NaN,NaN,...,245,146,2.0,1.0,3,G221,PRD,None,None,None
928119,701239235,G221,SUZUKA,2019-03-05,2019-03-05,08:00:00,08:00:00.000,480.0,NaN,NaN,...,1,392,3.0,1.0,4,G221,PRD,None,None,None
314206,701241831,G221,SUZUKA,2019-03-06,2019-03-06,08:00:00,08:00:00.000,480.0,NaN,NaN,...,2,393,4.0,1.0,5,G221,PRD,None,None,None


## Train Radnom Forrest Regressor

In [ ]:
# Necessary Imports
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from collections import Counter

# Step 1: Clean the data by dropping rows with missing EQUIPMENT_DESC
df_with_desc = df_unplanned.dropna(subset=['EQUIPMENT_DESC'])
df_missing_desc = df_unplanned[df_unplanned['EQUIPMENT_DESC'].isna()]

# Step 2: Define the correct categorical columns
categorical_columns = ['Plant', 'Process', 'Subprocess', 'Product_Line', 'Machine']

# Step 3: One-Hot Encode the categorical columns
encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
X_encoded = encoder.fit_transform(df_with_desc[categorical_columns])

# Step 4: Label encode the target variable (EQUIPMENT_DESC)
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(df_with_desc['EQUIPMENT_DESC'])

# Step 5: Remove classes with fewer than 5 instances (to avoid SMOTE issues)
class_counts = Counter(y_encoded)
valid_classes = [cls for cls, count in class_counts.items() if count >= 5]

# Filter out rows corresponding to classes with fewer than 5 instances
mask = [y in valid_classes for y in y_encoded]
X_encoded_filtered = X_encoded[mask]
y_encoded_filtered = y_encoded[mask]

# Step 6: Check distribution of the classes after filtering
class_distribution = Counter(y_encoded_filtered)
print(f"Class distribution after filtering: {class_distribution}")

# Step 7: Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X_encoded_filtered, y_encoded_filtered, test_size=0.3, random_state=42, stratify=y_encoded_filtered
)

# Step 8: Dynamically set k_neighbors based on the smallest class size in the training set
train_class_distribution = Counter(y_train)
smallest_class_count = min(train_class_distribution.values())

# Set k_neighbors to a minimum value of 1 if the smallest class has fewer samples
k_neighbors_value = max(1, min(5, smallest_class_count - 1))  # Ensure k_neighbors <= smallest class size

# Step 9: Apply SMOTE to balance the classes in the training set
smote = SMOTE(random_state=42, k_neighbors=k_neighbors_value)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

# Step 10: Check the distribution after SMOTE
print(f"Class distribution after SMOTE: {Counter(y_train_smote)}")

# Step 11: Train the RandomForestClassifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(X_train_smote, y_train_smote)

print("Model training complete.")

Class distribution after filtering: Counter({317: 3120, 478: 1049, 382: 1043, 390: 848, 316: 794, 327: 782, 475: 773, 391: 684, 452: 579, 509: 487, 397: 354, 550: 327, 405: 316, 321: 312, 474: 289, 453: 284, 393: 273, 66: 262, 281: 259, 406: 235, 462: 226, 392: 221, 69: 219, 64: 210, 309: 209, 552: 198, 63: 198, 404: 197, 516: 195, 515: 171, 402: 167, 746: 160, 277: 151, 476: 148, 588: 147, 331: 146, 68: 138, 415: 136, 477: 134, 481: 133, 542: 124, 563: 121, 541: 120, 429: 118, 279: 117, 269: 116, 289: 116, 448: 115, 520: 111, 519: 110, 333: 104, 522: 102, 234: 101, 357: 96, 65: 93, 285: 90, 290: 83, 335: 81, 487: 81, 414: 79, 110: 77, 134: 77, 416: 74, 446: 73, 140: 72, 394: 70, 430: 68, 590: 67, 631: 64, 503: 63, 294: 63, 380: 62, 549: 60, 556: 59, 362: 59, 457: 59, 227: 54, 196: 53, 326: 53, 119: 52, 525: 50, 70: 50, 489: 50, 517: 48, 407: 48, 514: 47, 461: 46, 423: 43, 447: 41, 363: 39, 358: 39, 359: 38, 420: 38, 544: 36, 286: 36, 662: 35, 141: 34, 328: 34, 554: 33, 488: 33, 643: 3

## Evaluate

In [ ]:
# Step 12: Get filtered target names
filtered_target_names = label_encoder.inverse_transform(valid_classes)

# Step 13: Make predictions on the test set and evaluate the model
y_pred = rf_classifier.predict(X_test)

# Ensure the target names match the filtered classes
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred, target_names=filtered_target_names))

## Predict

In [ ]:
# Step 14: Prepare the missing data for prediction
X_missing_encoded = encoder.transform(df_missing_desc[categorical_columns])

# Step 15: Use the model to predict the missing EQUIPMENT_DESC values
y_missing_pred = rf_classifier.predict(X_missing_encoded)

# Step 16: Convert the predictions back to the original EQUIPMENT_DESC values
predicted_descriptions = label_encoder.inverse_transform(y_missing_pred)

# Step 17: Fill the missing EQUIPMENT_DESC values in the original dataframe
df_unplanned.loc[df_unplanned['EQUIPMENT_DESC'].isna(), 'EQUIPMENT_DESC'] = predicted_descriptions

# Final dataframe with filled EQUIPMENT_DESC values
print(df_unplanned)

In [ ]:
# Check the column names in the df_unplanned dataframe
print(df_unplanned.columns)

## Time To Failure

In [ ]:
# Ensure the EXECUTION_FINISH_DATE is in datetime format
df_unplanned['EXECUTION_FINISH_DATE'] = pd.to_datetime(df_unplanned['EXECUTION_FINISH_DATE'])

# Sort the dataframe by EQUIPMENT_DESC and EXECUTION_FINISH_DATE
df_unplanned = df_unplanned.sort_values(by=['EQUIPMENT_DESC', 'EXECUTION_FINISH_DATE'])

# Create the TIME_TO_FAILURE column
df_unplanned['TIME_TO_FAILURE'] = df_unplanned.groupby('EQUIPMENT_DESC')['EXECUTION_FINISH_DATE'].diff().dt.days

## Splitting By Plant

In [ ]:
# Split the dataframe by 'PRODUCTION_LOCATION'
df_unplanned_MONZA = df_unplanned[df_unplanned['PRODUCTION_LOCATION'] == "MONZA"]
df_unplanned_ROMA = df_unplanned[df_unplanned['PRODUCTION_LOCATION'] == "ROMA"]
df_unplanned_COTA = df_unplanned[df_unplanned['PRODUCTION_LOCATION'] == "COTA"]
df_unplanned_SILVERSTONE = df_unplanned[df_unplanned['PRODUCTION_LOCATION'] == "SILVERSTONE"]
df_unplanned_MONACO = df_unplanned[df_unplanned['PRODUCTION_LOCATION'] == "MONACO"]
df_unplanned_SUZUKA = df_unplanned[df_unplanned['PRODUCTION_LOCATION'] == "SUZUKA"]